# Incomplete

In [1]:
from bleak import BleakScanner, BleakClient
from pylsl import StreamInfo, StreamOutlet
import asyncio, struct, time

In [2]:
HR_SERVICE_UUID = "0000180d-0000-1000-8000-00805f9b34fb"     # Heart Rate
HR_MEAS_CHAR    = "00002a37-0000-1000-8000-00805f9b34fb"     # Heart Rate Measurement

# LSL stream (BPM + RR interval in seconds; RR=-1.0 if absent)
info = StreamInfo(name="HeartRate", type="HR", channel_count=2,
                  nominal_srate=0.0, channel_format="float32", source_id="ble_hr")
info.desc().append_child("channels") \
    .append_child("channel").append_child_value("label", "BPM").parent() \
    .parent().append_child("channel").append_child_value("label", "RR_sec")
outlet = StreamOutlet(info, chunk_size=0, max_buffered=3600)


In [4]:
devices = await BleakScanner.discover(return_adv=True)
target = None
for d, adv in devices.values():
    print(d)
    if HR_SERVICE_UUID.lower() in [s.lower() for s in adv.service_uuids or []]:
        target = d; break
print(target)

E0:FD:13:FD:26:E1: TELEMATIC-461463
50:0B:56:E1:E5:73: None
20:27:34:15:DE:63: None
E7:1E:7E:A8:9A:EF: MATTER
CA:EC:2F:95:AE:62: Govee_H6079_AE62
71:F9:8F:B6:41:03: None
80:8A:BD:87:49:87: None
DE:11:2F:94:61:3C: None
0C:B4:FE:0F:01:F6: None
CB:26:B5:10:2C:71: None
09:BA:E7:6A:2F:BA: None
2D:66:87:AD:20:00: None
69:C9:15:B6:71:AA: None
D8:D2:61:E7:0A:CF: net
43:C4:5C:46:B6:16: None
53:0D:77:EE:F9:97: None
D0:49:3C:D6:A6:8A: None
FF:FF:4A:03:A5:02: None
2F:23:2F:29:AF:23: None
A4:C1:38:25:E5:F8: Minger_H6001_E5F8
00:1C:C2:9D:FA:7C: DREOtf11b7B
6F:5F:9C:A1:C2:54: None
3E:1B:6E:4A:36:2F: None
EC:1D:AC:33:95:BB: None
14:9D:2C:FF:F1:C0: None
None


In [ ]:
import asyncio
from bleak import BleakClient, BleakScanner
from bleak.backends.device import BLEDevice # Import for type hinting



# HR_SERVICE_UUID = "0000180d-0000-1000-8000-00805f9b34fb"

async def discover_and_list_data():
    target: BLEDevice = None
    devices = await BleakScanner.discover(return_adv=True)
    
    # Use your existing discovery logic to find the target
    for d, adv in devices.values():
        if HR_SERVICE_UUID.lower() in [s.lower() for s in adv.service_uuids or []]:
            target = d; break
        print(target)
    if target:
        print(f"Target device found: {target.address} ({target.name})")
        # Connect to the device using BleakClient as an async context manager
        async with BleakClient(target.address) as client:
            print(f"Connected: {client.is_connected}")
            
            # The available services are automatically enumerated upon connection
            services = client.services
            
            print("\n--- Available Services and Characteristics ---")
            for service in services:
                print(f"Service: {service.uuid} ({service.description})")
                
                for char in service.characteristics:
                    properties = ", ".join(char.properties)
                    print(f"  Characteristic: {char.uuid} (Props: {properties})")
                    
                    # Optional: Print Descriptors for more detail
                    for descriptor in char.descriptors:
                        print(f"    Descriptor: {descriptor.uuid} ({descriptor.description})")

    else:
        print("Target device not found.")

# asyncio.run(discover_and_list_data())
import nest_asyncio
nest_asyncio.apply()
asyncio.run(discover_and_list_data())


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Target device not found.


In [37]:
import asyncio
from bleak import BleakClient
from bleak.backends.characteristic import BleakGATTCharacteristic
data_store = []
# global data_store
# Replace with your actual device address
ADDRESS = target.address

# A simple callback function to handle incoming notification data
def data_handler(characteristic: BleakGATTCharacteristic, data: bytearray):
    """Prints the raw data received from a characteristic."""
    # The UUID identifies which stream of data is coming in.
    data_store.append(data.hex())
    print(f"Data : [{characteristic.uuid}] Data: {data.hex()} ({len(data)} bytes)")
    # For a smartwatch, this raw 'data' is the collected sensor information 
    # (e.g., heart rate, step count packet, etc.) and would require reverse 
    # engineering to decode into human-readable values.

async def collect_smartwatch_data(address: str):
    print(f"Attempting to connect to {address}...")
    
    # Use the async context manager to ensure connection/disconnection
    async with BleakClient(address) as client:
        if not client.is_connected:
            print("Failed to connect.")
            return

        print("Successfully connected. Subscribing to all 'notify' characteristics...")

        # Find and subscribe to all 'notify' or 'indicate' characteristics
        for service in client.services:
            for char in service.characteristics:
                uuid = char.uuid
                
                # 'notify' and 'indicate' are the mechanisms for streaming data
                if "notify" in char.properties or "indicate" in char.properties:
                    try:
                        # Pass the characteristic and the handler function
                        await client.start_notify(char, data_handler)
                        print(f"  -> Subscribed to NOTIFY/INDICATE on {uuid}")
                    except Exception as e:
                        print(f"  -> FAILED to subscribe to {uuid}: {e}")

        print("\n--- Listening for live data for 30 seconds... (Press Ctrl+C to stop) ---\n")
        
        # Keep the program running for a duration to collect streaming data
        await asyncio.sleep(30) 
        
        # Clean up by stopping notifications (good practice)
        for service in client.services:
            for char in service.characteristics:
                if "notify" in char.properties or "indicate" in char.properties:
                    try:
                        await client.stop_notify(char)
                    except Exception:
                        pass # Ignore errors on stopping
        
        print("\nNotification stopped. Disconnecting.")


if __name__ == "__main__":
    try:
        asyncio.run(collect_smartwatch_data(ADDRESS))
    except KeyboardInterrupt:
        print("\nData collection stopped by user.")
    except RuntimeError as e:
        # Fallback for environments like Jupyter or already running loops
        print(f"\nRuntimeError detected: {e}. Trying nest_asyncio fix...")
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(collect_smartwatch_data(ADDRESS))

Attempting to connect to D6:32:17:D0:B9:7D...


BleakDeviceNotFoundError: Device with address D6:32:17:D0:B9:7D was not found.